<a href="https://colab.research.google.com/github/sanmeshh/pytorch_learning/blob/7.-ANN_on_gpu/ANN_fashion_gpu_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device:{device}')

device:cuda


In [8]:
#reproducibility
torch.manual_seed(108)

In [9]:
df=pd.read_csv("/content/fashion-mnist_train.csv")

In [10]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
#scaling the features values are like 0,142,213
X_train=X_train/255.0
X_test=X_test/255.0

In [14]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00392157, 0.        ,
        0.        ]])

In [15]:
#custom dataset
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)#remember float for features
    self.labels=torch.tensor(labels,dtype=torch.long)#and long for labels
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [16]:
#create train_dataset object
train_dataset=CustomDataset(X_train,y_train)


In [17]:
len(train_dataset)

48000

In [18]:
train_dataset[3]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0078, 0.0118, 0.0118, 0.0000, 0.0000, 0.0000, 0.0000, 0.0980, 0.4392,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0078,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1059, 0.4824, 0.8000, 0.8196,
         0.8863, 0.1804, 0.0

In [19]:
test_dataset=CustomDataset(X_test,y_test)

In [20]:
#dividing data into batches
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=False,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [21]:
#define NN class

class MyNN(nn.Module):
  def __init__(self,num_features):

    super().__init__()#imp step
    self.model=nn.Sequential(
        nn.Linear(num_features,128),#num_features=784
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
        )

  def forward(self,x):
    return self.model(x)


In [22]:
#set learning rate and epochs
epochs=100
lr=0.1

In [23]:
#model instance
model=MyNN(X_train.shape[1])
model=model.to(device)
#loss func
criterion=nn.CrossEntropyLoss()#take cares of the softmax as well

#optimizer
optimizer=optim.SGD(model.parameters(),lr=lr)

In [24]:
#training loop
for epoch in range(epochs):
  TotalEpochLoss=0
  for batch_features,batch_labels in train_loader:

    #move data to gpu
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)

    #forward pass
    outputs=model(batch_features)

    #loss
    loss=criterion(outputs,batch_labels)

    #backward pass
    optimizer.zero_grad()
    loss.backward()


    #update grads
    optimizer.step()

    #calc total loss of each epoch
    TotalEpochLoss+=loss.item()

  avg_loss=TotalEpochLoss/len(train_loader)

  print(f'Epoch:{epoch+1},Loss:{avg_loss}')







Epoch:1,Loss:0.636908844401439
Epoch:2,Loss:0.4298365565141042
Epoch:3,Loss:0.3830028742899497
Epoch:4,Loss:0.3556590295135975
Epoch:5,Loss:0.3344840477233132
Epoch:6,Loss:0.31603712886571883
Epoch:7,Loss:0.3022157291769981
Epoch:8,Loss:0.28928424706558387
Epoch:9,Loss:0.27923643589019775
Epoch:10,Loss:0.27046489987522365
Epoch:11,Loss:0.2596766291434566
Epoch:12,Loss:0.25052355827142797
Epoch:13,Loss:0.24262235800921916
Epoch:14,Loss:0.23486402109637858
Epoch:15,Loss:0.22696322744463882
Epoch:16,Loss:0.22017169364045064
Epoch:17,Loss:0.21340462515875697
Epoch:18,Loss:0.21139589868485928
Epoch:19,Loss:0.20192216572910548
Epoch:20,Loss:0.1999077203286191
Epoch:21,Loss:0.19283261846130093
Epoch:22,Loss:0.18760060214002927
Epoch:23,Loss:0.18581434925148885
Epoch:24,Loss:0.18413989468229314
Epoch:25,Loss:0.1761975788573424
Epoch:26,Loss:0.17254575016225376
Epoch:27,Loss:0.1687877958845347
Epoch:28,Loss:0.1619419232942164
Epoch:29,Loss:0.16085062274026374
Epoch:30,Loss:0.1630639963094145
Ep

In [25]:
# set model to evaluation mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [27]:
#evaluation code
total=0
correct=0

#we dont want to calculate gradients during evaluation
with torch.no_grad():

  for batch_features,batch_labels in test_loader:
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)

    outputs=model(batch_features)


    _ , predicted=torch.max(outputs,1)

    #getting the total no of samples by adding samples of each batch
    total=total+batch_labels.shape[0]
    # print(total)

    correct+=(predicted==batch_labels).sum().item()

  print(correct/total)








0.8750833333333333


In [ ]:
 len(test_loader)